In [4]:
import psycopg2
conn = psycopg2.connect(host='localhost',database='barbers', user='postgres', password='postgres')
conn.autocommit = True
cur = conn.cursor()

query0 = """
ALTER TABLE goodsoperation
DROP COLUMN "masterId",
DROP COLUMN "masterBonusSum";
"""

query1 = """
ALTER TABLE goodsoperation
ADD COLUMN "employeeBonusSum" FLOAT(2),
ADD COLUMN "employeeId" INT;
"""

query2 = """
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'goodsoperation'
"""

query3 = """
truncate table goodsoperation
"""

query4 = """
select * from goodsoperation
"""

cur.execute(query1)
# rows = cur.fetchall()
# for row in rows:
#     print (row)
cur.close()
conn.close()
print('success')

success


In [2]:
import json
a = "{\"id\": 36}"
loaded = json.loads(a)
type(loaded)

dict